In [83]:
import numpy as np
from numpy.fft import fft2, ifft2, fftshift, ifftshift
from numpy import angle, real
from numpy import exp, abs, pi, sqrt
import matplotlib.pyplot as plt
import cv2

In [84]:

def load_img(path, bw=False, width=200, height=200):
  img = cv2.imread(path)
  if bw:
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  if width and height:
    try:
        img = cv2.resize(img, (height, width), interpolation=cv2.INTER_LINEAR)
    except Exception as e:
        print(path)
    
  return img

In [107]:

dataset_list = []
dataset_size = 219
for i in range(1, dataset_size+1):
    img = load_img(f'./dataset/{i}.PNG', width=2532/6, height=1170/6)
    dataset_list.append(img)

dataset = np.array(dataset_list)

./dataset/1.PNG
./dataset/2.PNG
./dataset/3.PNG
./dataset/4.PNG
./dataset/5.PNG
./dataset/6.PNG
./dataset/7.PNG
./dataset/8.PNG
./dataset/9.PNG
./dataset/10.PNG
./dataset/11.PNG
./dataset/12.PNG
./dataset/13.PNG
./dataset/14.PNG
./dataset/15.PNG
./dataset/16.PNG
./dataset/17.PNG
./dataset/18.PNG
./dataset/19.PNG
./dataset/20.PNG
./dataset/21.PNG
./dataset/22.PNG
./dataset/23.PNG
./dataset/24.PNG
./dataset/25.PNG
./dataset/26.PNG
./dataset/27.PNG
./dataset/28.PNG
./dataset/29.PNG
./dataset/30.PNG
./dataset/31.PNG
./dataset/32.PNG
./dataset/33.PNG
./dataset/34.PNG
./dataset/35.PNG
./dataset/36.PNG
./dataset/37.PNG
./dataset/38.PNG
./dataset/39.PNG
./dataset/40.PNG
./dataset/41.PNG
./dataset/42.PNG
./dataset/43.PNG
./dataset/44.PNG
./dataset/45.PNG
./dataset/46.PNG
./dataset/47.PNG
./dataset/48.PNG
./dataset/49.PNG
./dataset/50.PNG
./dataset/51.PNG
./dataset/52.PNG
./dataset/53.PNG
./dataset/54.PNG
./dataset/55.PNG
./dataset/56.PNG
./dataset/57.PNG
./dataset/58.PNG
./dataset/59.PNG
./data

In [108]:
# to try with small section
dataset1 = np.array(dataset_list[:2])
frames = np.array(dataset1) / 255.
numFrames = frames.shape[0]
height = frames.shape[1]
width = frames.shape[2]

In [112]:
def magnifyChange(im1, im2, magnificationFactor):
    
    # find phase shift in frequency domain
    im1Dft = fft2(im1)
    im2Dft = fft2(im2)
    phaseShift = angle(im2Dft / im1Dft)
    # magnify the phase change in frequency domain
    magnifiedDft = np.abs(im2Dft) * np.exp(1j*(magnificationFactor * phaseShift + angle(im1Dft)))
    # what does the magnified phase change cause in image space?
    magnified = ifft2(magnifiedDft).real
    
    return magnified

In [113]:
magnified_two = magnifyChange(dataset_list[0], dataset_list[1], 10)

/var/folders/px/4z6skxwj33gc4lyfccvmncth0000gn/T/ipykernel_95330/8634067.py:6: RuntimeWarning: divide by zero encountered in true_divide
  phaseShift = angle(im2Dft / im1Dft)
/var/folders/px/4z6skxwj33gc4lyfccvmncth0000gn/T/ipykernel_95330/8634067.py:6: RuntimeWarning: invalid value encountered in true_divide
  phaseShift = angle(im2Dft / im1Dft)


In [117]:
magnified_two.shape
cv2.imwrite(f'./results/img_trying.PNG', magnified_two)

True

In [110]:
def magnify_frames(frames, magnificationFactor=10, sigma=13, alpha=0.5):
  # we will magnify windows of the video and aggregate the results
  magnified = np.zeros_like(frames)

  # meshgrid for computing Gaussian window
  X, Y = np.meshgrid(np.arange(width), np.arange(height))

  # iterate over windows of the frames
  xRange = list(range(0, width, 2*sigma))
  yRange = list(range(0, height, 2*sigma))
  numWindows = len(xRange) * len(yRange)
  windowIndex = 1

  for y in yRange:
      for x in xRange:
          for channelIndex in range(3): # RGB channels
              for frameIndex in range(numFrames):
                  
                  # create windowed frames
                  diff = ( x - X ) ** 2 + ( y - Y ) ** 2
                  gaussianMask = np.exp(-diff / (2*sigma**2)) / (1 / np.sqrt(2*np.pi * sigma ** 2))
                  windowedFrames = gaussianMask * frames[frameIndex,:,:,channelIndex]
              
                  # initialize moving average of phase for current window/channel
                  if frameIndex == 0:
                      windowAveragePhase = angle(fft2(windowedFrames))
                  
                  windowDft = fft2(windowedFrames)
                  
                  # compute phase shift and constrain to [-pi, pi] since
                  # angle space wraps around
                  windowPhaseShift = angle(windowDft) - windowAveragePhase
                  windowPhaseShift[windowPhaseShift > pi] = windowPhaseShift[windowPhaseShift > pi] - 2 * pi
                  windowPhaseShift[windowPhaseShift < -pi] = windowPhaseShift[windowPhaseShift < -pi] + 2 * pi
                  
                  # magnify phase shift
                  windowMagnifiedPhase = windowAveragePhase + windowPhaseShift * magnificationFactor
                  
                  # go back to image space
                  windowMagnifiedDft = abs(windowDft) * np.exp(windowMagnifiedPhase*1j) 
                  windowMagnified = abs(ifft2(windowMagnifiedDft))
                  
                  # update moving average
                  windowPhaseUnwrapped = windowAveragePhase + windowPhaseShift
                  windowAveragePhase = alpha * windowAveragePhase + (1 - alpha) * windowPhaseUnwrapped
                  
                  # aggregate
                  magnified[frameIndex,:,:,channelIndex] = magnified[frameIndex,:,:,channelIndex] + windowMagnified
          
  return magnified

In [111]:
magnified = magnify_frames(frames)

KeyboardInterrupt: 

In [ ]:
outputs = magnified / np.max(magnified)
for channelIndex in range(3):
    originalFrame = frames[0,:,:,channelIndex]
    magnifiedFrame = outputs[0,:,:,channelIndex]
    scale = np.std(originalFrame[:]) / np.std(magnifiedFrame[:])
    originalMean = np.mean(originalFrame[:])
    magnifiedMean = np.mean(magnifiedFrame[:])
    outputs[:,:,:,channelIndex] = magnifiedMean + scale * (outputs[:,:,:,channelIndex] - magnifiedMean)

outputs = np.clip(outputs, 0, 1)

In [ ]:
def imshow(im, cmap='gray', clip=False):
    # clip image from 0-1
    if clip:
        im = np.clip(im, 0, 1)
    plt.imshow(im, cmap=cmap)

In [ ]:
for i in range(outputs.shape[0]):
  cv2.imwrite(f'./results/img{i}.PNG', (outputs[i]*255).astype(np.uint8))
  # if i % 2 == 0:
  #   plt.figure()
  #   plt.subplot(121)
  # else:
  #   plt.subplot(122)
  # imshow(outputs[i])